In [2]:
# TODO:

# 1.
# Create a summary for each of the six topics:
# For each topic, take the heading, title and all the headings AND titles of the all the articles of the topic's subsections.
# Each time a question is asked, ask LLM to provide the topic for the answer.
# NB! This costs 2x more than it could

# 2.
# Take the same from above and embed it so each topic's distance could be calculated for the Question.


# 3.
# Download the text for the articles and save these in a file. Have python create an automatic file structure.
# Embed all the articles and save the embeddings next to the article text.


In [37]:
import numpy as np

with open('embeddings/subsection_embeddings.npy', 'rb') as f:
    subsection_embeddings = np.load(f)

with open('embeddings/article_embeddings.npy', 'rb') as f:
    article_embeddings = np.load(f)

In [80]:
import openai
# openai.api_key = ''

In [38]:
EMBEDDING_MODEL = "text-embedding-ada-002"


def get_embedding(text: str, model: str = EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
        model=model,
        input=text
    )
    return result["data"][0]["embedding"]

In [63]:
question = 'Can I transfer moeny to a business in Ukraine?'
question_embedding = get_embedding(question)


In [76]:
import json

with open("scraped-data/index.json", "r") as f:
    data = json.load(f)

def return_content(article_id):
    counter = 0
    for section in data:
        for subsection in section["subsections"]:
            for article in subsection["articles"]:
                if counter != article_id:
                    counter += 1
                    continue
                with open(article['folder_path'] + '/content.md', 'r') as f:
                    article_md = f.read()
                items = [section["title"], section["heading"], article_md]
                content = '\n\n'.join(items)
                return content, 'wise.com' + article['link']



In [0]:
import numpy as np

# 	$0.002 / 1K tokens
# 4,096 tokens = $0.008192

def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.

    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))


article_similarities = []

for article_nr, article_embedding in enumerate(article_embeddings):
    similarity = vector_similarity(question_embedding, article_embedding)
    article_similarities.append((similarity, article_nr))

article_similarities = sorted(article_similarities, reverse=True)


In [89]:
import tiktoken
MODEL = "gpt-3.5-turbo"
tokenizer = tiktoken.encoding_for_model(MODEL)

articles = [return_content(article_similarities[i][1]) for i in range(5)]

top_five_similar_articles = {url: dict(content=md, tokens=len(tokenizer.encode(md))) for md, url in articles}

token_count = 0
articles_up_to_2000_tokens = {}
for url, article in top_five_similar_articles.items():
    if token_count + article['tokens'] < 2000:
        token_count += article['tokens']
        articles_up_to_2000_tokens[url] = article
    else:
        break

len(articles_up_to_2000_tokens)

4

In [90]:
md = '\n\n'.join([article['content'] for article in articles_up_to_2000_tokens.values()])

In [91]:
prompt = '''
You are a customer support agent at Wise, the money transfer company. Your job is to answer the customer by using only the info provided here. You will get a customer's question on one line,
with the prefix "CUSTOMER: "
On the rest of the lines, you will get relevant information to answer the question. It starts with the prefix "CONTEXT: ".
Behave like you are the source of the information, while drawing answers only from the info provided. If your are unable to answer the question using the provided context, say 'I don't know'.
'''

In [92]:

messages = [{"role": "system", "content": prompt}, {"role": "user", "content": 'CUSTOMER:  \n' + question}, {"role": "system", "content": 'CONTEXT: \n' + md},]
chat = openai.ChatCompletion.create(
    model=MODEL,
    messages=messages
)
reply = chat.choices[0].message.content

reply

"Yes, you can transfer money to a private user of PrivatBank in Ukraine. However, Wise can't make transfers to any businesses in Ukraine."

In [93]:
messages.append({"role" : "assistant", "content": reply})
messages.append({"role": "user", "content": 'CUSTOMER:  \n' + 'How about Turkey?'})

chat = openai.ChatCompletion.create(
    model=MODEL,
    messages=messages
)
reply = chat.choices[0].message.content

reply

'Yes, you can transfer money to a business that is registered in Turkey. Wise allows bank transfers from Turkey only for registered businesses.'